In [1]:
####
# ML Homework
# Abishek Ganesh
####
sqlContext
from pyspark.sql.functions import udf
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.pipeline import Pipeline
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import PCA

from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier

In [2]:
read_data = sqlContext.read.csv("/FileStore/tables/f5rw9ku71493048580496/winequality_white-d3c3e.csv", header = True, sep=';', inferSchema=True)
print read_data.dtypes, len(read_data.dtypes)
read_data.show()

In [3]:
# Define Metrics 
from pyspark.mllib.evaluation import MulticlassMetrics
def printMetrics(predictions_and_labels):
    metrics = MulticlassMetrics(predictions_and_labels)
    print 'Precision of True ', metrics.precision(1)
    print 'Precision of False', metrics.precision(0)
    print 'Recall of True    ', metrics.recall(1)
    print 'Recall of False   ', metrics.recall(0)
    print 'F-1 Score         ', metrics.fMeasure()
    print 'Confusion Matrix\n', metrics.confusionMatrix().toArray()

In [5]:
def val_check(x):
  x = int(x)
  if x>=0 and x < 6:
    return 0
  if x > 6 and x <=10:
    return 2
  if x == 6:
    return 1
balance_class = udf(lambda x : val_check(x) ,IntegerType())
  
#1 = original
#2 = class Balance
#3 = correlation removed
#4 = Normalization
#5 = Std Scalar
#6 = PCA
#7 = Sampling 

typ = 7


#1 = original
if typ == 1:
  input_data = read_data.withColumn('label', read_data.quality)
  va  = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"], outputCol="features")
  pipeline = Pipeline(stages= [va])

#2 = class Balance 
if typ == 2:
  input_data = read_data.withColumn('label', balance_class(read_data.quality))
  va  = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"], outputCol="features")
  pipeline = Pipeline(stages= [va])

#3 = Feature Selection
if typ == 3:
  input_data = read_data.withColumn('label', balance_class(read_data.quality))
  va  = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides",  "total sulfur dioxide", "pH", "sulphates", "alcohol"], outputCol="features") # free sulfur dioxide and density removed for clearing association  
  pipeline = Pipeline(stages= [va])

#4 = Normalization
if typ == 4:
  input_data = read_data.withColumn('label', balance_class(read_data.quality))
  va  = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"], outputCol="features1")
  normalizer = Normalizer(inputCol="features1", outputCol="features", p=1.0)
  pipeline = Pipeline(stages= [va, normalizer])
  
#5 Std Scalar
if typ == 5:
  input_data = read_data.withColumn('label', balance_class(read_data.quality))
  va  = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"], outputCol="features1")
  scalar = StandardScaler(inputCol="features1", outputCol="features", withStd=True, withMean=True)
  pipeline = Pipeline(stages= [va, scalar])

#PCA
if typ == 6:
  input_data = read_data.withColumn('label', balance_class(read_data.quality))
  va  = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"], outputCol="features1")
  pca = PCA(k=2, inputCol="features1", outputCol="features")
  pipeline = Pipeline(stages= [va, pca])

if typ == 7:
  input_data = read_data.withColumn('label', balance_class(read_data.quality)).sampleBy('label', fractions={0:0.3, 1:0.3, 2:0.4})
  va  = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"], outputCol="features")
  pipeline = Pipeline(stages= [va])



input_data.cache()
print input_data.dtypes , len(input_data.dtypes)
print pipeline.getStages()

In [6]:
dataset = pipeline.fit(input_data).transform(input_data).select('features', 'label')
dataset.show(5)
dataset.dtypes

In [7]:
# Set Data for train and set
(train, test)  = dataset.randomSplit([0.70, 0.30] ,seed = 100)
print train.count()
print test.count()

In [8]:
####
#Logistic Regression Part 1 : Train
####

lr = LogisticRegression(labelCol='label', featuresCol='features', maxIter=1000000)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
lrcvModel = cv.fit(train)

In [9]:
####
#Logistic Regression Part 2 : Test
####

predictions = lrcvModel.transform(test)

evaluator = MulticlassClassificationEvaluator( metricName='accuracy' )
print evaluator.getMetricName(), evaluator.evaluate(predictions)
evaluator = MulticlassClassificationEvaluator( metricName='f1' )
print evaluator.getMetricName(), evaluator.evaluate(predictions)

if typ !=1:
  printMetrics(predictions.select('prediction', 'label' ).rdd.map(lambda x : (x[0],float(x[1]))))

In [10]:
####
#Random Forest Classifier Part 1 : Train
####

rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
rfcvModel = cv.fit(train)

In [11]:
####
#Random Forest Classifier Part 2 : Test
####
predictions = rfcvModel.transform(test)
evaluator = MulticlassClassificationEvaluator( metricName='accuracy' )
print evaluator.getMetricName(), evaluator.evaluate(predictions)
evaluator = MulticlassClassificationEvaluator( metricName='f1' )
print evaluator.getMetricName(), evaluator.evaluate(predictions)
if typ !=1:
  printMetrics(predictions.select('prediction', 'label' ).rdd.map(lambda x : (x[0],float(x[1]))))

In [12]:
####
#Decision Tree Classifier Part 1 : Train
####

dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1,2,6,10])
             .addGrid(dt.maxBins, [20,40,80])
             .build())

cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
dtcvModel = cv.fit(train)

In [13]:
####
#Decision Tree Classifier Part 2 : Test
####
predictions = dtcvModel.transform(test)
evaluator = MulticlassClassificationEvaluator( metricName='accuracy' )
print evaluator.getMetricName(), evaluator.evaluate(predictions)
evaluator = MulticlassClassificationEvaluator( metricName='f1' )
print evaluator.getMetricName(), evaluator.evaluate(predictions)
if typ !=1:
  printMetrics(predictions.select('prediction', 'label' ).rdd.map(lambda x : (x[0],float(x[1]))))

In [14]:
####
#SVM in the form of oneVrest Part 1: Train
####
lr = LogisticRegression(maxIter=100000, tol=1E-6, fitIntercept=True)
ovr = OneVsRest(classifier=lr)
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1,2,6,10])
             .addGrid(dt.maxBins, [20,40,80])
             .build())
cv = CrossValidator(estimator=ovr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
svmcvModel = cv.fit(train)

In [15]:
####
#SVM in the form of oneVrest Part 2 :  Test
####
predictions =  svmcvModel.transform(test)
evaluator = MulticlassClassificationEvaluator( metricName='accuracy' )
print evaluator.getMetricName(), evaluator.evaluate(predictions)
evaluator = MulticlassClassificationEvaluator( metricName='f1' )
print evaluator.getMetricName(), evaluator.evaluate(predictions)
if typ !=1:
  printMetrics(predictions.select('prediction', 'label' ).rdd.map(lambda x : (x[0],float(x[1]))))